# Stage 1: Setup - Step 2: Validate Configuration

This step validates that all required configurations and files are in place.

**Purpose:**
- Check for required configuration files
- Validate file formats
- Verify data integrity
- Report any issues

In [1]:
# Step tracking initialization
%run /workspace/system/helpers/03_StepTracker.ipynb
%run /workspace/system/helpers/04_Display.ipynb

# Initialize step tracker
step = StepTracker(
    cycle_name="{{CYCLE_NAME}}",
    stage_num=1,
    step_num=2,
    idempotent=False,  # This step should not be re-run once completed
    auto_start=True
)

if not step.can_execute:
    print("Step cannot be executed. Check the message above.")

✅ Configuration loaded - Base path: /workspace
✅ Database connection successful
✅ Step Tracker loaded
✅ Display utilities loaded
❌ Active cycle '{{CYCLE_NAME}}' not found
Step cannot be executed. Check the message above.


## Validation Logic

In [2]:
if step.can_execute:
    try:
        print_header("Configuration Validation")
        
        from pathlib import Path
        import json
        
        # Get cycle information
        active = get_active_cycle()
        cycle_dir = Path(f"/workspace/workflows/Active_{active['cycle_name']}")
        files_dir = cycle_dir / "files"
        
        validation_results = {
            'passed': [],
            'failed': [],
            'warnings': []
        }
        
        # 1. Check for Excel configuration files
        print("\n1. Checking for Excel configuration files...")
        excel_dir = files_dir / "excel_configuration"
        if excel_dir.exists():
            excel_files = list(excel_dir.glob("*.xlsx")) + list(excel_dir.glob("*.xls"))
            if excel_files:
                print(f"   ✅ Found {len(excel_files)} Excel file(s)")
                for f in excel_files:
                    print(f"      - {f.name}")
                validation_results['passed'].append(f"Excel files: {len(excel_files)} found")
            else:
                print("   ⚠️ No Excel files found")
                validation_results['warnings'].append("No Excel configuration files")
        else:
            excel_dir.mkdir(parents=True, exist_ok=True)
            print(f"   ℹ️ Created directory: {excel_dir}")
            validation_results['warnings'].append("Excel directory created")
        
        # 2. Check for extracted configurations
        print("\n2. Checking for extracted configurations...")
        extract_dir = files_dir / "extracted_configurations"
        if extract_dir.exists():
            json_files = list(extract_dir.glob("*.json"))
            if json_files:
                print(f"   ✅ Found {len(json_files)} JSON configuration(s)")
                validation_results['passed'].append(f"JSON configs: {len(json_files)} found")
            else:
                print("   ℹ️ No extracted configurations yet")
                validation_results['warnings'].append("No extracted configurations")
        else:
            extract_dir.mkdir(parents=True, exist_ok=True)
            print(f"   ℹ️ Created directory: {extract_dir}")
            validation_results['warnings'].append("Extract directory created")
        
        # 3. Check database tables
        print("\n3. Checking database tables...")
        required_tables = ['irp_cycle', 'irp_stage', 'irp_step', 'irp_batch', 'irp_job']
        for table in required_tables:
            try:
                count = execute_scalar(f"SELECT COUNT(*) FROM {table}")
                print(f"   ✅ Table {table}: {count} rows")
                validation_results['passed'].append(f"Table {table} exists")
            except Exception as e:
                print(f"   ❌ Table {table}: Not found")
                validation_results['failed'].append(f"Table {table} missing")
        
        # 4. Summary
        print("\n" + "=" * 60)
        print("Validation Summary:")
        print(f"  ✅ Passed: {len(validation_results['passed'])}")
        print(f"  ⚠️ Warnings: {len(validation_results['warnings'])}")
        print(f"  ❌ Failed: {len(validation_results['failed'])}")
        
        # Determine if validation passed
        if validation_results['failed']:
            raise Exception(f"Validation failed with {len(validation_results['failed'])} errors")
        
        # Mark step as completed
        step.complete_run(validation_results)
        
        print("\n✅ VALIDATION COMPLETE")
        if validation_results['warnings']:
            print("\nWarnings to address:")
            for warning in validation_results['warnings']:
                print(f"  ⚠️ {warning}")
        
    except Exception as e:
        step.fail_run(str(e))
        print(f"\n❌ Validation failed: {e}")
        raise

## Next Steps

In [3]:
print_header("Next Steps")

print("""
Based on the validation results:

1. If Excel files are missing:
   - Add configuration files to: files/excel_configuration/
   
2. If database tables are missing:
   - Run Database_Admin.ipynb to initialize
   
3. Once all validations pass:
   - Proceed to Stage 2 notebooks
""")

 Next Steps

Based on the validation results:

1. If Excel files are missing:
   - Add configuration files to: files/excel_configuration/
   
2. If database tables are missing:
   - Run Database_Admin.ipynb to initialize
   
3. Once all validations pass:
   - Proceed to Stage 2 notebooks

